In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://books.toscrape.com/"
res = requests.get(BASE_URL)
soup = BeautifulSoup(res.text, "html.parser")

Categories_list_Path = soup.find("ul", class_="nav nav-list")
Category_name = Categories_list_Path.find_all("a")[1].text.replace("\n", "").strip()  # Correct index for category

with open("books.toscrape.csv", mode="w", encoding="utf-8") as fd:
    fd.write("Category_Name, Book_Name, Book_Price, Book_Rating, Stock_Status\n")

    page_len =soup.find("li" , attrs= {"class" : "current"}).text.replace("\n" , "").replace("Page of ", "").strip()
    
    for page in range (page_len ):
        
        BASE_URL = "https://books.toscrape.com/"
        res = requests.get(BASE_URL)
        soup = BeautifulSoup(res.text, "html.parser")
        Categories_list_Path = soup.find("ul", class_="nav nav-list")
        page_number = 1
  
        page_url = f"{Category_url.replace('index.html', '')}page-{page_number}.html"
        res_category = requests.get(page_url)
        soup_category = BeautifulSoup(res_category.text, "html.parser")
        
        books = soup_category.find_all("article", class_="product_pod")
        
        if not books:
            break

        for book in books:
            Book_name = book.find("h3").find("a").attrs['title']
            Book_Price = book.find("p", class_="price_color").text
            stock_status = book.find("p", class_="instock availability").text.replace("\n", "").strip()
            rating = book.find("p", class_="star-rating")['class'][1]
            rating_map = {
                'One': '1 star',
                'Two': '2 stars',
                'Three': '3 stars',
                'Four': '4 stars',
                'Five': '5 stars'
            }
            Book_Rating = rating_map.get(rating, "Unknown")
            
            # Write data to the CSV file
            fd.write(f"\"{Category_name}\", \"{Book_name.replace(',', ' ').strip()}\", \"{Book_Price}\", \"{Book_Rating}\", \"{stock_status}\"\n")
        
        page_number += 1


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


UrL = "https://books.toscrape.com"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
books = []

    for book in soup.find_all('article', class_='product_pod'):
        title = book.find('h3').find('a').get('title')
        price = book.find('p', class_='price_color').text
        rating = book.find('p', class_='star-rating')['class'][1]  # Gets the rating class (e.g., 'One', 'Two', etc.)
        availability = book.find('p', class_='instock availability').text.strip()
        
        books.append({
            'Title': title,
            'Price': price,
            'Rating': rating,
            'Availability': availability
        })
    
    return books

# Function to scrape multiple pages
def scrape_books(base_url):
    page_number = 1
    all_books = []
    
    while True:
        url = f"{base_url}/catalogue/page-{page_number}.html"
        books = fetch_page_data(url)
        
        if not books:
            break
        
        all_books.extend(books)
        page_number += 1
    
    return all_books

# Base URL of the website
base_url = "https://books.toscrape.com"

# Scraping the books
books_data = scrape_books(base_url)

# Saving the data to a CSV file
df = pd.DataFrame(books_data)
df.to_csv('books_data.csv', index=False)

print(f"Scraped {len(books_data)} books.")
